<a href="https://colab.research.google.com/github/Sudhanshu20004-9/AI-tech-Lab1/blob/main/AITAtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:


!unzip /content/obj.zip -d /content/


Archive:  /content/obj.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/obj.zip or
        /content/obj.zip.zip, and cannot find /content/obj.zip.ZIP, period.


In [ ]:
# Install necessary libraries
!pip install tensorflow opencv-python pillow
!pip install git+https://github.com/zzh8829/yolov3-tf2.git

import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from yolov3.tf import YOLOv3
from tensorflow.keras.models import load_model

# Step 1: Prepare the Dataset
def convert_annotation(xml_file, image_width, image_height):
    """
    Converts the annotation in Pascal VOC XML format to YOLO format.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bboxes = []

    for obj in root.iter('object'):
        name = obj.find('name').text
        if name == "vehicle":  # Filter only vehicle objects
            # Get the bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # Convert to YOLO format
            x_center = (xmin + xmax) / 2.0 / image_width
            y_center = (ymin + ymax) / 2.0 / image_height
            width = (xmax - xmin) / float(image_width)
            height = (ymax - ymin) / float(image_height)

            # Append the vehicle annotation in YOLO format (class id, x_center, y_center, width, height)
            bboxes.append(f"0 {x_center} {y_center} {width} {height}")  # 0 is the vehicle class

    return bboxes

# Convert annotations for all images
image_width = 1280  # Example image width (replace with actual)
image_height = 720  # Example image height (replace with actual)

annotations_dir = '/content/dataset/annotations/'  # Path to annotations (XML)
image_dir = '/content/dataset/images/'  # Path to images

for xml_file in os.listdir(annotations_dir):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(annotations_dir, xml_file)
        image_path = os.path.join(image_dir, xml_file.replace('.xml', '.jpg'))

        yolo_annotations = convert_annotation(xml_path, image_width, image_height)

        # Save the YOLO annotations to a text file
        txt_file = os.path.join(annotations_dir, xml_file.replace('.xml', '.txt'))
        with open(txt_file, 'w') as f:
            for annotation in yolo_annotations:
                f.write(f"{annotation}\n")

# Step 2: Set Up YOLOv3 Model
from yolov3.tf import YOLOv3
from yolov3.utils import load_weights

# Initialize YOLOv3
yolo = YOLOv3(classes=1)  # We have 1 class (vehicle)

# Load pre-trained weights (COCO dataset, for example)
weights_path = '/content/yolov3.weights'
load_weights(yolo, weights_path)

# Step 3: Prepare the Data Generator
def create_data_generator(image_dir, annotations_dir, batch_size=32):
    """
    Generates batches of images and labels for YOLOv3 model.
    """
    image_paths = []
    label_paths = []
    for filename in os.listdir(annotations_dir):
        if filename.endswith('.txt'):
            image_paths.append(os.path.join(image_dir, filename.replace('.txt', '.jpg')))
            label_paths.append(os.path.join(annotations_dir, filename))

    def generator():
        while True:
            for image_path, label_path in zip(image_paths, label_paths):
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (416, 416))
                image = np.array(image, dtype=np.float32) / 255.0  # Normalize

                with open(label_path, 'r') as f:
                    labels = f.readlines()

                boxes = []
                for label in labels:
                    parts = label.strip().split()
                    class_id, x_center, y_center, width, height = map(float, parts)
                    boxes.append([x_center, y_center, width, height])
                boxes = np.array(boxes)
                yield (image, boxes)

    return generator()

# Step 4: Train YOLO Model
batch_size = 16
train_generator = create_data_generator(image_dir, annotations_dir, batch_size)

# Compile the model (adjust learning rate and optimizer as needed)
yolo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='yolo_loss')

# Train the model (set the number of epochs)
yolo.fit(train_generator, steps_per_epoch=100, epochs=50)

# Save the trained model
yolo.save('vehicle_detection_yolo_model.h5')

# Step 5: Run Inference on New Images
def detect_vehicles(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (416, 416))  # Resize to YOLO input size
    image_normalized = np.array(image_resized, dtype=np.float32) / 255.0  # Normalize

    # Get model predictions
    predictions = yolo.predict(np.expand_dims(image_normalized, axis=0))

    # Process predictions (boxes, classes, and scores)
    boxes, scores, classes, valid_detections = predictions

    # Draw bounding boxes for detected vehicles
    for i in range(valid_detections[0]):
        box = boxes[0][i]
        score = scores[0][i]
        class_id = int(classes[0][i])

        if score > 0.5 and class_id == 0:  # Filter by confidence and class ID
            x_center, y_center, width, height = box
            x1 = int((x_center - width / 2) * image.shape[1])
            y1 = int((y_center - height / 2) * image.shape[0])
            x2 = int((x_center + width / 2) * image.shape[1])
            y2 = int((y_center + height / 2) * image.shape[0])

            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show the image with detected vehicles
    cv2.imshow('Vehicle Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Run inference on a test image
detect_vehicles('/content/test_image.jpg')
